In [1]:
import numpy as np
import pandas as pd

In [2]:
# load data
train = pd.read_csv('tweets_train.csv')
test = pd.read_csv('tweets_test.csv')


In [3]:
# map hanle to category
def allocate_category(handle):
    if (handle == 'HillaryClinton'):
        return 0
    else:
        return 1
train['category'] = train['handle'].map(lambda handle: allocate_category(handle))

In [4]:
# create naive bayse classifyier based on text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])


In [5]:
# use cross-validation to approximate classifier generallity error
from sklearn.model_selection import cross_val_score
scores = cross_val_score(text_clf, train.text, train.category, cv=10)
print('average cross-validation error on train-set is: ', scores.mean())

('average cross-validation error on train-set is: ', 0.81215581972387318)


In [6]:
# predicting results
_ = text_clf.fit(train.text,train.category)
test['category'] = text_clf.predict(test.text)
def category_2_candidate(category):
    if (category == 0):
        return 'HillaryClinton'
    else:
        return 'realDonaldTrump'
test['handle'] = test['category'].map(lambda category: category_2_candidate(category))

In [7]:
# saving results
del test['category'] # deleting helper coloumn 
test.to_csv('Basic_tweets_test.csv',encoding='utf8',index = False)

In [8]:
# validating results
test_from_save = pd.read_csv('Basic_tweets_test.csv')
test_from_save.head()

,id,handle,text,is_retweet,original_author,time,in_reply_to_screen_name,is_quote_status,lang,retweet_count,favorite_count
0,7.340000e+17,HillaryClinton,#MichaelBrown would have been 20 years old tod...,True,LSD_Esq,2016-05-20T18:07:08,NaN,False,en,594,1096
1,7.340000e+17,HillaryClinton,"Congratulations on becoming a U.S. citizen, Al...",False,NaN,2016-05-20T17:24:12,NaN,False,en,1701,4239
2,7.340000e+17,HillaryClinton,We need a president who will unite leaders aro...,False,NaN,2016-05-20T17:12:52,NaN,False,en,1817,3577
3,7.340000e+17,HillaryClinton,"Dear Congress,\r\n\r\nLet's get this done.\r\n...",False,NaN,2016-05-20T16:21:13,NaN,False,en,2530,6012
4,7.340000e+17,realDonaldTrump,Failing @NYTimes will always take a good story...,False,NaN,2016-05-20T16:11:21,NaN,False,en,3750,12372
